In [1]:
import pandas as pd
import os 
import sys
import numpy as np
import sklearn
import time
import copy
from collections import defaultdict
import seaborn as sns
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import torch.nn.functional as F
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from sklearn.decomposition import PCA

In [2]:
#create single pose dataset for dnn model
class SinglePoseDataset(Dataset):
    
    def __init__(self, n_frames=5, pca_features=26):
        
        # get data and label from csvget 
        dataset, labels = SinglePoseDataset.ReadPoseData(n_frames)
        
        
        self.X = dataset
        self.y = labels
        
        self.X = self.apply_pca(pca_features)
        print('dataset shape:',self.X.shape, 'labels shape', self.y.shape)
        
    # read dataset from csv file
    @staticmethod
    def ReadPoseData(n_frames):
        
        # get csv file path
        curr_dir = os.getcwd()
        csv_file_path = os.path.join(curr_dir, 'data/drop_res.csv')
        
        # list for storing data and labels
        data  = []
        label = []
        
        # lenth of sequence
        #n_frames = SinglePoseDataset.n_frames
        
        # read csv file
        KP_df = pd.read_csv(csv_file_path)
        #print(KP_df['pos_class'])
        # convert pos_class to categories
        KP_df['pos_class'] = KP_df['pos_class'].astype('category')
        KP_df['pos_class'] = KP_df['pos_class'].cat.codes

        # skipping (0-3) colomns , return values of all rows and columns from 4 to last
        features = KP_df.iloc[:,5:].values
        #return values of pose_class 
        pose_class = KP_df['pos_class'].values
        # normalize keypoints 
        SinglePoseDataset.normalize_min_(features)
        # append multiple rows to create a sequence of data
        for i in range(0,features.shape[0]-n_frames,1):
            data.append(features[i:i+n_frames,...])
            label_sequence = pose_class[i:i+n_frames]
            #print('label sequence:',label_sequence)
            unique, counts = np.unique(label_sequence, return_counts=True)
            #print('label:', unique[np.argmax(counts)])
            label.append(unique[np.argmax(counts)])
            
        data , label =  np.array(data, dtype = np.float), np.array(label, dtype = np.int_)
        
        return data , label
    
    # min-max normalization to scale the x, y coordinates in range (0-1) 
    @staticmethod
    def normalize_min_(pose:np.ndarray):
        pose = pose.reshape(len(pose),-1,2)
        for i in range(len(pose)):
            xmin = np.min(pose[i,:,0]) 
            ymin = np.min(pose[i,:,1])
            xlen = np.max(pose[i,:,0]) - xmin
            ylen = np.max(pose[i,:,1]) - ymin

            if(xlen==0): pose[i,:,0]=0
            else:
                pose[i,:,0] -= xmin 
                pose[i,:,0] /= xlen

            if(ylen==0): pose[i,:,1]=0
            else:
                pose[i,:,1] -= ymin
                pose[i,:,1] /= ylen
        return pose
    
    def apply_pca(self, pca_features):
        
        #apply pca for dimensionalty reduction
        pca = PCA(n_components=pca_features)
        self.reshape_features()
        return pca.fit_transform(self.X)
        
        
    # number of rows in the dataset
    def __len__(self):
        
        return len(self.X)
        
    # get a row at an index
    def __getitem__(self, idx):
        
        data  = torch.tensor(self.X[idx], dtype=torch.float) 
        label = torch.tensor(self.y[idx], dtype=torch.long)
        
        return [data,label]
    
    # get indexes for train and test rows
    def get_splits(self, n_test = 0.2, n_valid=0.2):
        
        # determine sizes 
        test_size = round(n_test * len(self.X))
        valid_size = round(n_valid * len(self.X))
        train_size = len(self.X)-(test_size+valid_size)
        print(train_size, valid_size, test_size)
        # calculate the split 
        return random_split(self, [train_size, valid_size, test_size])
    
    def get_class_labels(self):
        
        labels = ["Fall","Stand", "Tie"]
        
        return labels
    
    def reshape_features(self):
        self.X = self.X.reshape(-1, self.X.shape[1]*self.X.shape[2])

In [3]:
class DNN(torch.nn.Module):
    
    def __init__(self, input_dim, class_num, initrange=0.5, drop_p=0.7):
        
        super().__init__()
        self.fc1 = torch.nn.Linear(input_dim,1024)
        self.bn1 = torch.nn.BatchNorm1d(1024)
        self.fc2 = torch.nn.Linear(1024, 512)
        self.bn2 = torch.nn.BatchNorm1d(512)
        self.fc3 = torch.nn.Linear(512,256)
        self.bn3 = torch.nn.BatchNorm1d(256)
        self.fc4 = torch.nn.Linear(256,128)
        self.bn4 = torch.nn.BatchNorm1d(128)
        self.fc5 = torch.nn.Linear(128, class_num)
        self.class_num = class_num
        self.init_weights(initrange)
        self.p = drop_p
    
    def init_weights(self, initrange):
        
        self.fc1.weight.data.uniform_(-initrange, initrange)
        self.fc1.bias.data.zero_()
        self.fc2.weight.data.uniform_(-initrange, initrange)
        self.fc2.bias.data.zero_()
        self.fc3.weight.data.uniform_(-initrange, initrange)
        self.fc3.bias.data.zero_()
        self.fc4.weight.data.uniform_(-initrange, initrange)
        self.fc4.bias.data.zero_()
        self.fc5.weight.data.uniform_(-initrange, initrange)
        self.fc5.bias.data.zero_()
    
    def forward(self, _input):
        
        _bn1 = self.bn1(self.fc1(_input))
        _fc1 = F.dropout(F.relu(_bn1),p=self.p) 
        
        _bn2 = self.bn2(self.fc2(_bn1))
        _fc2 = F.dropout(F.relu(_bn2), p=self.p)
        
        _bn3 = self.bn3(self.fc3(_fc2))
        _fc3 = F.dropout(F.relu(_bn3), p=0.5)
        
        _bn4 = self.bn4(self.fc4(_fc3))
        _fc4 = F.dropout(F.relu(_bn4), p=0.5)
        
        _fc5 = self.fc5(_fc4)
        
        output = F.softmax(_fc5, dim=1)
        
        return output

In [4]:
class DNN_(torch.nn.Module):
    
    def __init__(self, input_dim, class_num, initrange=0.5):
        
        super().__init__()
        self.fc1 = torch.nn.Linear(input_dim,128)
        self.bn1 = torch.nn.BatchNorm1d(128)
        self.fc2 = torch.nn.Linear(128, 64)
        self.bn2 = torch.nn.BatchNorm1d(64)
        self.fc3 = torch.nn.Linear(64,16)
        self.bn3 = torch.nn.BatchNorm1d(16)
        self.fc4 = torch.nn.Linear(16, class_num)
        self.class_num = class_num
        self.init_weights(initrange)
    
    def init_weights(self, initrange):
        
        self.fc1.weight.data.uniform_(-initrange, initrange)
        self.fc1.bias.data.zero_()
        self.fc2.weight.data.uniform_(-initrange, initrange)
        self.fc2.bias.data.zero_()
        self.fc3.weight.data.uniform_(-initrange, initrange)
        self.fc3.bias.data.zero_()
        self.fc4.weight.data.uniform_(-initrange, initrange)
        self.fc4.bias.data.zero_()
    
    def forward(self, _input):
        
        _fc1 = F.relu(self.fc1(_input))
        
        _bn1 = self.bn1(_fc1)
        
        _fc2 = F.relu(self.fc2(_bn1))
        _bn2 = self.bn2(_fc2)
        
        _fc3 = F.relu(self.fc3(_bn1))
        _bn3 = self.bn3(_fc3)
        
        _fc4 = self.fc4(_bn3)
        
        output = F.softmax(_fc4, dim=1)
        
        return output


In [5]:
def prepare_data():
    
    # load pose dataset
    dataset = SinglePoseDataset(n_frames=5, pca_features=26)
    
    # reshape from N,10,34 to N,340
    #dataset.reshape_features()
    
    # calculate split size
    train,  valid, test = dataset.get_splits()
    
    # prepare data loaders
    train_dl =  DataLoader(train, batch_size=64, shuffle=True)
    valid_dl  = DataLoader(valid, batch_size=64, shuffle=False)
    test_dl  =  DataLoader(test, batch_size=64, shuffle=False)
    
    return {'train':train_dl, 'valid':valid_dl, 'test':test_dl}, {'train':train, 'valid':valid,'test':test}

In [6]:
def save_model(model, optimizer, loss, acc, epoch, save_path):
    
    base_dir = os.path.basename(os.getcwd())
    
    if base_dir =='train':
        parent_dir = os.path.dirname(os.getcwd())
        os.chdir(parent_dir)
        if not os.path.exists(save_path):
            os.makedirs(save_path)
    
    print('SAVING EPOCH %d'%epoch)
    
    SAVE_FILE = os.path.join(save_path,'epoch_%d.pth'%epoch)
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            'acc':  acc,
            }, SAVE_FILE)    

In [7]:
def train_model(model, dataloaders, dataset_sizes, num_epochs=3000):
    
    
    history = defaultdict(list)
    
    #get train and val dataloaders
    #dataloaders, dataset_sizes = prepare_data()

    # define the optimization
    criterion = torch.nn.CrossEntropyLoss().to(device)
    optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=500, gamma=0.1)
    
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    conf_train = 0.0
    conf_valid = 0.0
    # enumerate over epochs
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs-1))
        print('-'*10)
        
        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train() # St model to training mode
            
            else:
                model.eval() # Set model to evaluate mode
                
            running_loss = 0.0
            running_corrects = 0.0
            # Initialize the prediction and label lists(tensors)
            pred_tensor  = torch.zeros(0,dtype=torch.long, device='cpu')
            class_tensor = torch.zeros(0,dtype=torch.long, device='cpu')
            
            # enumerate over mini_batch
            for i, (inputs, targets)  in enumerate(dataloaders[phase]):
                inputs  = inputs.to(device)
                targets = targets.to(device)
                
                # clear the parameter gradients
                optimizer.zero_grad()
                
                # forward
                # track history if only in train pahse
                with torch.set_grad_enabled(phase=='train'):
                    
                    # compute model outputs
                    outputs = model(inputs)
                    
                    # calculate outputs
                    _, preds = torch.max(outputs, dim=1)
                    
                    # calculate the loss
                    loss = criterion(outputs, targets)
                    
                    # backward + optimize only ig in training phase
                    if phase == 'train':
                        # calculate gradient
                        loss.backward()
                    
                        # update model weights
                        optimizer.step()
                
                #statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds==targets)
                
                # Append batch prediction results
                pred_tensor = torch.cat([pred_tensor,preds.view(-1).cpu()])
                class_tensor  = torch.cat([class_tensor,targets.view(-1).cpu()])
                
            
            if phase=='train':
                scheduler.step()
            
            #epoch loss and accuracy
            epoch_loss = running_loss / dataset_sizes[phase].__len__()
            epoch_acc  = running_corrects.double() / dataset_sizes[phase].__len__()
            history[phase].append((epoch_loss, epoch_acc, ))
            
            # Confusion matrix
            conf_mat = confusion_matrix(class_tensor.numpy(), pred_tensor.numpy())
            # Per-class accuracy
            #per_class_accuracy = np.round(100*conf_mat.diagonal()/conf_mat.sum(1),4)
            #Precision, Recall, F1_Score
            precision = precision_score(class_tensor.numpy(), pred_tensor.numpy(), average=None)
            recall = recall_score(class_tensor.numpy(), pred_tensor.numpy(), average=None)
            f_score = f1_score(class_tensor.numpy(), pred_tensor.numpy(), average=None)
            
            print('{} : Loss: {:.4f}, Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            print('{} : Confusion Matrix: {}'.format(phase, conf_mat))
            print('{} : Precision per class: {}'.format(phase, np.round(precision,4)))
            print('{} : Recall per class: {}'.format(phase, np.round(recall,4)))
            print('{} : F1_Score per class: {}'.format(phase, np.round(f_score,4)))
            print()
            
            if phase== 'valid' and epoch_acc>best_acc:
                best_acc = epoch_acc
                epoch_ = epoch
                loss_ = epoch_loss
                conf_valid = conf_mat
                #best_model_wts = copy.deepcopy(model.state_dict())
                #save_model(model, optimizer, loss_, best_acc, epoch_, save_path=r'checkpoints_fallmodel/act_dnnSingle_5')
            
            if phase== 'train' and epoch_acc>best_acc:
                conf_train = conf_mat
               
                
        print()
    
    time_elapsed = time.time() - since

    
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed //60, time_elapsed %60))
    print('Best val Acc: {:4f}'.format(best_acc))
    
    # load best model weights
    #model.load_state_dict(best_model_wts)
    save_model(model, optimizer, epoch_loss, best_acc, epoch, save_path=r'checkpoints_fallmodel/act_dnnSingle_5')
    
    return history, model, conf_train, conf_valid

In [8]:
#POSE_JOINT_SIZE = 34
#n_frames = 5
pca_features = 26
DNN_model = DNN(input_dim=pca_features,class_num=3).to(device)

In [9]:
#get test dataloaders
dataloaders, dataset_sizes = prepare_data()
history, model, conf_train, conf_valid = train_model(DNN_model, dataloaders, dataset_sizes,num_epochs=3000)

Index(['LShoulder_x', 'LShoulder_y', 'RShoulder_x', 'RShoulder_y', 'LElbow_x',
       'LElbow_y', 'RElbow_x', 'RElbow_y', 'LWrist_x', 'LWrist_y', 'RWrist_x',
       'RWrist_y', 'LHip_x', 'LHip_y', 'RHip_x', 'RHip_y', 'LKnee_x',
       'LKnee_y', 'Rknee_x', 'Rknee_y', 'LAnkle_x', 'LAnkle_y', 'RAnkle_x',
       'RAnkle_y'],
      dtype='object')
dataset shape: (18872, 26) labels shape (18872,)
11324 3774 3774
Epoch 0/2999
----------
train : Loss: 0.8905, Acc: 0.6597
train : Confusion Matrix: [[2739  794  840]
 [ 406 2826  617]
 [ 512  685 1905]]
train : Precision per class: [0.749  0.6564 0.5666]
train : Recall per class: [0.6263 0.7342 0.6141]
train : F1_Score per class: [0.6822 0.6932 0.5894]

valid : Loss: 0.7235, Acc: 0.8484
valid : Confusion Matrix: [[1296   91   73]
 [  62 1154   74]
 [ 114  158  752]]
valid : Precision per class: [0.8804 0.8225 0.8365]
valid : Recall per class: [0.8877 0.8946 0.7344]
valid : F1_Score per class: [0.884  0.857  0.7821]


Epoch 1/2999
----------
trai

train : Loss: 0.5907, Acc: 0.9613
train : Confusion Matrix: [[4222   70   81]
 [  54 3711   84]
 [  73   76 2953]]
train : Precision per class: [0.9708 0.9621 0.9471]
train : Recall per class: [0.9655 0.9641 0.952 ]
train : F1_Score per class: [0.9681 0.9631 0.9495]

valid : Loss: 0.5869, Acc: 0.9642
valid : Confusion Matrix: [[1420   28   12]
 [  11 1255   24]
 [  29   31  964]]
valid : Precision per class: [0.9726 0.9551 0.964 ]
valid : Recall per class: [0.9726 0.9729 0.9414]
valid : F1_Score per class: [0.9726 0.9639 0.9526]


Epoch 15/2999
----------
train : Loss: 0.5899, Acc: 0.9612
train : Confusion Matrix: [[4235   64   74]
 [  50 3709   90]
 [  68   93 2941]]
train : Precision per class: [0.9729 0.9594 0.9472]
train : Recall per class: [0.9684 0.9636 0.9481]
train : F1_Score per class: [0.9707 0.9615 0.9476]

valid : Loss: 0.5835, Acc: 0.9666
valid : Confusion Matrix: [[1410   35   15]
 [  13 1254   23]
 [  21   19  984]]
valid : Precision per class: [0.9765 0.9587 0.9628]
val

train : Loss: 0.5825, Acc: 0.9678
train : Confusion Matrix: [[4247   60   66]
 [  54 3734   61]
 [  49   75 2978]]
train : Precision per class: [0.9763 0.9651 0.9591]
train : Recall per class: [0.9712 0.9701 0.96  ]
train : F1_Score per class: [0.9737 0.9676 0.9596]

valid : Loss: 0.5782, Acc: 0.9727
valid : Confusion Matrix: [[1413   24   23]
 [  11 1266   13]
 [  13   19  992]]
valid : Precision per class: [0.9833 0.9672 0.965 ]
valid : Recall per class: [0.9678 0.9814 0.9688]
valid : F1_Score per class: [0.9755 0.9742 0.9669]


Epoch 30/2999
----------
train : Loss: 0.5801, Acc: 0.9714
train : Confusion Matrix: [[4273   46   54]
 [  40 3748   61]
 [  44   79 2979]]
train : Precision per class: [0.9807 0.9677 0.9628]
train : Recall per class: [0.9771 0.9738 0.9603]
train : F1_Score per class: [0.9789 0.9707 0.9616]

valid : Loss: 0.5791, Acc: 0.9727
valid : Confusion Matrix: [[1429   25    6]
 [  10 1267   13]
 [  26   23  975]]
valid : Precision per class: [0.9754 0.9635 0.9809]
val

train : Loss: 0.5750, Acc: 0.9762
train : Confusion Matrix: [[4272   56   45]
 [  31 3765   53]
 [  37   48 3017]]
train : Precision per class: [0.9843 0.9731 0.9685]
train : Recall per class: [0.9769 0.9782 0.9726]
train : F1_Score per class: [0.9806 0.9756 0.9706]

valid : Loss: 0.5721, Acc: 0.9791
valid : Confusion Matrix: [[1433   18    9]
 [  11 1266   13]
 [  17   11  996]]
valid : Precision per class: [0.9808 0.9776 0.9784]
valid : Recall per class: [0.9815 0.9814 0.9727]
valid : F1_Score per class: [0.9812 0.9795 0.9755]


Epoch 45/2999
----------
train : Loss: 0.5748, Acc: 0.9762
train : Confusion Matrix: [[4291   38   44]
 [  31 3768   50]
 [  54   52 2996]]
train : Precision per class: [0.9806 0.9767 0.9696]
train : Recall per class: [0.9812 0.979  0.9658]
train : F1_Score per class: [0.9809 0.9778 0.9677]

valid : Loss: 0.5725, Acc: 0.9785
valid : Confusion Matrix: [[1436   16    8]
 [   7 1268   15]
 [  21   14  989]]
valid : Precision per class: [0.9809 0.9769 0.9773]
val

train : Loss: 0.5713, Acc: 0.9803
train : Confusion Matrix: [[4306   38   29]
 [  32 3779   38]
 [  49   37 3016]]
train : Precision per class: [0.9815 0.9805 0.9783]
train : Recall per class: [0.9847 0.9818 0.9723]
train : F1_Score per class: [0.9831 0.9812 0.9753]

valid : Loss: 0.5728, Acc: 0.9783
valid : Confusion Matrix: [[1428   25    7]
 [   3 1271   16]
 [  14   17  993]]
valid : Precision per class: [0.9882 0.968  0.9774]
valid : Recall per class: [0.9781 0.9853 0.9697]
valid : F1_Score per class: [0.9831 0.9766 0.9735]


Epoch 60/2999
----------
train : Loss: 0.5721, Acc: 0.9794
train : Confusion Matrix: [[4303   41   29]
 [  30 3779   40]
 [  38   55 3009]]
train : Precision per class: [0.9844 0.9752 0.9776]
train : Recall per class: [0.984  0.9818 0.97  ]
train : F1_Score per class: [0.9842 0.9785 0.9738]

valid : Loss: 0.5690, Acc: 0.9820
valid : Confusion Matrix: [[1445    9    6]
 [   6 1269   15]
 [  15   17  992]]
valid : Precision per class: [0.9857 0.9799 0.9793]
val

...

train : Loss: 0.5540, Acc: 0.9975
train : Confusion Matrix: [[4362    5    6]
 [   4 3843    2]
 [   3    8 3091]]
train : Precision per class: [0.9984 0.9966 0.9974]
train : Recall per class: [0.9975 0.9984 0.9965]
train : F1_Score per class: [0.9979 0.9975 0.9969]

valid : Loss: 0.5564, Acc: 0.9947
valid : Confusion Matrix: [[1453    3    4]
 [   1 1284    5]
 [   4    3 1017]]
valid : Precision per class: [0.9966 0.9953 0.9912]
valid : Recall per class: [0.9952 0.9953 0.9932]
valid : F1_Score per class: [0.9959 0.9953 0.9922]


Epoch 2940/2999
----------
train : Loss: 0.5541, Acc: 0.9974
train : Confusion Matrix: [[4362    6    5]
 [   4 3839    6]
 [   3    6 3093]]
train : Precision per class: [0.9984 0.9969 0.9965]
train : Recall per class: [0.9975 0.9974 0.9971]
train : F1_Score per class: [0.9979 0.9971 0.9968]

valid : Loss: 0.5569, Acc: 0.9947
valid : Confusion Matrix: [[1456    2    2]
 [   2 1284    4]
 [   5    5 1014]]
valid : Precision per class: [0.9952 0.9946 0.9941]
v

train : Loss: 0.5541, Acc: 0.9975
train : Confusion Matrix: [[4368    3    2]
 [   4 3842    3]
 [   6   10 3086]]
train : Precision per class: [0.9977 0.9966 0.9984]
train : Recall per class: [0.9989 0.9982 0.9948]
train : F1_Score per class: [0.9983 0.9974 0.9966]

valid : Loss: 0.5561, Acc: 0.9952
valid : Confusion Matrix: [[1455    1    4]
 [   1 1283    6]
 [   2    4 1018]]
valid : Precision per class: [0.9979 0.9961 0.9903]
valid : Recall per class: [0.9966 0.9946 0.9941]
valid : F1_Score per class: [0.9973 0.9953 0.9922]


Epoch 2955/2999
----------
train : Loss: 0.5547, Acc: 0.9966
train : Confusion Matrix: [[4362    5    6]
 [   4 3838    7]
 [   6   10 3086]]
train : Precision per class: [0.9977 0.9961 0.9958]
train : Recall per class: [0.9975 0.9971 0.9948]
train : F1_Score per class: [0.9976 0.9966 0.9953]

valid : Loss: 0.5576, Acc: 0.9936
valid : Confusion Matrix: [[1455    3    2]
 [   1 1282    7]
 [   6    5 1013]]
valid : Precision per class: [0.9952 0.9938 0.9912]
v

train : Loss: 0.5540, Acc: 0.9974
train : Confusion Matrix: [[4364    5    4]
 [   3 3841    5]
 [   5    7 3090]]
train : Precision per class: [0.9982 0.9969 0.9971]
train : Recall per class: [0.9979 0.9979 0.9961]
train : F1_Score per class: [0.9981 0.9974 0.9966]

valid : Loss: 0.5567, Acc: 0.9947
valid : Confusion Matrix: [[1453    4    3]
 [   0 1286    4]
 [   2    7 1015]]
valid : Precision per class: [0.9986 0.9915 0.9932]
valid : Recall per class: [0.9952 0.9969 0.9912]
valid : F1_Score per class: [0.9969 0.9942 0.9922]


Epoch 2970/2999
----------
train : Loss: 0.5548, Acc: 0.9965
train : Confusion Matrix: [[4361    6    6]
 [   5 3837    7]
 [   4   12 3086]]
train : Precision per class: [0.9979 0.9953 0.9958]
train : Recall per class: [0.9973 0.9969 0.9948]
train : F1_Score per class: [0.9976 0.9961 0.9953]

valid : Loss: 0.5575, Acc: 0.9942
valid : Confusion Matrix: [[1458    0    2]
 [   1 1283    6]
 [   6    7 1011]]
valid : Precision per class: [0.9952 0.9946 0.9921]
v

train : Loss: 0.5542, Acc: 0.9972
train : Confusion Matrix: [[4364    6    3]
 [   5 3842    2]
 [   5   11 3086]]
train : Precision per class: [0.9977 0.9956 0.9984]
train : Recall per class: [0.9979 0.9982 0.9948]
train : F1_Score per class: [0.9978 0.9969 0.9966]

valid : Loss: 0.5567, Acc: 0.9947
valid : Confusion Matrix: [[1456    1    3]
 [   1 1285    4]
 [   4    7 1013]]
valid : Precision per class: [0.9966 0.9938 0.9931]
valid : Recall per class: [0.9973 0.9961 0.9893]
valid : F1_Score per class: [0.9969 0.995  0.9912]


Epoch 2985/2999
----------
train : Loss: 0.5547, Acc: 0.9966
train : Confusion Matrix: [[4365    5    3]
 [   4 3837    8]
 [   8   10 3084]]
train : Precision per class: [0.9973 0.9961 0.9964]
train : Recall per class: [0.9982 0.9969 0.9942]
train : F1_Score per class: [0.9977 0.9965 0.9953]

valid : Loss: 0.5584, Acc: 0.9931
valid : Confusion Matrix: [[1454    4    2]
 [   0 1282    8]
 [   3    9 1012]]
valid : Precision per class: [0.9979 0.99   0.9902]
v

train : Loss: 0.5544, Acc: 0.9970
train : Confusion Matrix: [[4361    6    6]
 [   2 3841    6]
 [   4   10 3088]]
train : Precision per class: [0.9986 0.9959 0.9961]
train : Recall per class: [0.9973 0.9979 0.9955]
train : F1_Score per class: [0.9979 0.9969 0.9958]

valid : Loss: 0.5583, Acc: 0.9931
valid : Confusion Matrix: [[1453    3    4]
 [   0 1281    9]
 [   3    7 1014]]
valid : Precision per class: [0.9979 0.9923 0.9873]
valid : Recall per class: [0.9952 0.993  0.9902]
valid : F1_Score per class: [0.9966 0.9926 0.9888]


Training complete in 32m 52s
Best val Acc: 0.996820
SAVING EPOCH 2999


In [ ]:
train_acc  = []
train_loss = []
val_acc =  []
val_loss = []

for train_item, val_item in zip(history['train'],history['valid']):
    
    val_loss.append(val_item.__getitem__(0))
    val_acc.append(val_item.__getitem__(1).cpu().detach().numpy())
    
    train_loss.append(train_item.__getitem__(0))
    train_acc.append(train_item.__getitem__(1).cpu().detach().numpy())

In [ ]:
num_epochs=range(3000)
plt.figure(figsize=(16, 6))
plt.plot(num_epochs, train_acc, label='Training Accuracy')
plt.plot(num_epochs, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.savefig('5.acc.jpg')
plt.show()

In [ ]:
num_epochs=range(3000)
plt.figure(figsize=(16, 6))
plt.plot(num_epochs, train_loss, label='Training Loss')
plt.plot(num_epochs, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.savefig('5_loss.jpg')
plt.show()

In [ ]:
#set the size of figure 
plt.figure(figsize=(8,8))
#normalize each column (class) with total datapoints in that column  
conf_train = conf_train.astype('float')/conf_train.sum(axis=1)*100
#plot confusion matrix 
p=sns.heatmap(conf_train, xticklabels=['Fall','Stand','Tie'], yticklabels=['Fall','Stand','Tie'],
              cbar=False, annot=True, cmap='coolwarm',robust=True, fmt='.1f',annot_kws={'size':20})
plt.title('Training matrix: Actual labels Vs Predicted labels')
plt.savefig('conf_mat_10_train.png')

In [ ]:
#set the size of figure 
plt.figure(figsize=(8,8))
#normalize each column (class) with total datapoints in that column  
conf_valid = conf_valid.astype('float')/conf_valid.sum(axis=1)*100
#plot confusion matrix 
p=sns.heatmap(conf_valid, xticklabels=['Fall','Stand','Tie'], yticklabels=['Fall','Stand','Tie'],
              cbar=False, annot=True, cmap='coolwarm',robust=True, fmt='.1f',annot_kws={'size':20})
plt.title('Validation matrix: Actual labels vs Predicted labels')
plt.savefig('conf_mat_10_valid.png')

In [ ]:
def evaluate_model(model, dataloaders, dataset_sizes):
    #defaultdict to save list in dict
    history = defaultdict(list)
    criterion = torch.nn.CrossEntropyLoss().to(device)
    model.eval() # Set model to evaluate mode
    phase = 'test'            
    running_loss = 0.0
    running_corrects = 0.0
    # Initialize the prediction and label lists(tensors)
    pred_tensor  = torch.zeros(0,dtype=torch.long, device='cpu')
    class_tensor = torch.zeros(0,dtype=torch.long, device='cpu')

    # enumerate over mini_batch
    for i, (inputs, targets)  in enumerate(dataloaders['test']):
        inputs  = inputs.to(device)
        targets = targets.to(device)

        # compute model outputs
        outputs = model(inputs)

        # calculate outputs
        _, preds = torch.max(outputs, dim=1)

        # calculate the loss
        loss = criterion(outputs, targets)

        #statistics
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds==targets)

        # Append batch prediction results
        pred_tensor = torch.cat([pred_tensor,preds.view(-1).cpu()])
        class_tensor  = torch.cat([class_tensor,targets.view(-1).cpu()])
                
    #epoch loss and accuracy
    loss = running_loss / dataset_sizes['test'].__len__()
    acc  = running_corrects.double() / dataset_sizes['test'].__len__()
    history['test'].append((loss,acc))

    # Confusion matrix
    conf_mat = confusion_matrix(class_tensor.numpy(), pred_tensor.numpy())
    # Per-class accuracy
    per_class_accuracy = np.round(100*conf_mat.diagonal()/conf_mat.sum(1),4)
    #Precision, Recall, F1_Score
    precision = precision_score(class_tensor.numpy(), pred_tensor.numpy(), average='micro')
    recall = recall_score(class_tensor.numpy(), pred_tensor.numpy(), average='micro')
    f_score = f1_score(class_tensor.numpy(), pred_tensor.numpy(), average='micro')

    print('{} : Loss: {:.4f}, Acc: {:.4f}'.format(phase, loss, acc))
    print('{} : Confusion Matrix: {}, Per_class_accuracy: {}'.format(phase, conf_mat, per_class_accuracy))
    print('{} : Precision: {:.4f}, Recall: {:.4f}, F1_Score: {:.4f}'.format(phase, precision, recall, f_score))
    print()
            
    #set the size of figure 
    plt.figure(figsize=(8,8))
    #normalize each column (class) with total datapoints in that column  
    conf_mat = conf_mat.astype('float')/conf_mat.sum(axis=1)*100
    #plot confusion matrix 
    p=sns.heatmap(conf_mat, xticklabels=['Fall','Stand','Tie'], yticklabels=['Fall','Stand','Tie'],
              cbar=False, annot=True, cmap='coolwarm',robust=True, fmt='.1f',annot_kws={'size':20})
    plt.title('Test matrix: Actual labels Vs Predicted labels')
    plt.savefig('conf_mat_10_test.png')

In [ ]:
evaluate_model(model, dataloaders, dataset_sizes)

In [ ]:
Obtain actual data from the real environment through cameras or sensors